<a href="https://colab.research.google.com/github/nerson/bigdata_dissertacao_UFC/blob/main/furtos_veiculos_sp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


__Instalando as bibliotecas.__

In [ ]:
!pip install panda
!pip install numpy
!pip install geopandas
!pip install matplotlib
!pip install folium
!pip install shapely

__Como importar as bibliotecas.__

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import folium
from shapely.geometry import Point, LineString, Polygon
from folium.plugins import FastMarkerCluster
from folium.plugins import HeatMap

Baixando os dados dos municípios e da microrregião
do estado de São Paulo pelo site do IBGE.

https://www.ibge.gov.br/geociencias/organizacao-do-territorio/malhas-territoriais/15774-malhas.html

In [ ]:
mapas_municipios_sp = gpd.read_file('dados/SP_Municipios_2022.shp')

__Lendo os dados baixados.__

In [ ]:
mapas_municipios_sp.head()

__Selecionando apenas a linha com informações do município de Morro Agudo.__

In [ ]:
mapas_municipios_sp[mapas_municipios_sp['NM_MUN'] == 'Morro Agudo']

__Plotando o mapa do estado de São Paulo com seus municípios.__

In [ ]:
mapas_municipios_sp.plot(figsize=(12,10), facecolor='white', edgecolor='black')

plt.show()

__Criando um dataframe com informações do município de Morro Agudo.__

In [ ]:
mapas_municipio_morroagudo = mapas_municipios_sp[mapas_municipios_sp['NM_MUN'] == 'Morro Agudo']

In [ ]:
mapas_municipio_morroagudo.plot(figsize=(6,8), facecolor='white', edgecolor='black')

plt.show()

In [ ]:
mapas_municipio_morroagudo

__Gravando os dados do município de Morro Agudo em um arquivo.__

In [ ]:
mapas_municipio_morroagudo.to_file('dados/municipio_morro_agudo.json', driver='GeoJSON')

__Criando um dataframe com informações dos furtos de veículos.__

Download dos dados: https://www.ssp.sp.gov.br/transparenciassp/

In [ ]:
df_01 = pd.read_excel('dados/DadosBO_2022_12_furtos_veiculos.xlsx')
df_02 = pd.read_excel('dados/DadosBO_2023_1_furtos_veiculos.xlsx')

In [ ]:
df_furto_veiculos_bimestral_sp = pd.concat([df_01, df_02])

In [ ]:
df_furto_veiculos_bimestral_sp

__Filtrando os dados de roubos para o município de Morro Agudo.__

In [ ]:
df_furto_veiculos_bimestral_morro_agudo = df_furto_veiculos_bimestral_sp[df_furto_veiculos_bimestral_sp['CIDADE'] == 'MORRO AGUDO']

In [ ]:
df_furto_veiculos_bimestral_morro_agudo['CIDADE']

__Verificando a quantidade de dados nulos por coluna.__

In [ ]:
df_furto_veiculos_bimestral_morro_agudo[['LATITUDE', 'LONGITUDE']]

In [ ]:
df_antes = df_furto_veiculos_bimestral_morro_agudo.shape[0]
df_atual = df_furto_veiculos_bimestral_morro_agudo.dropna(subset=['LATITUDE', 'LONGITUDE']).shape[0]
print("{} linhas seriam removidas, o equivalente a {}% do dataframe".format(df_antes-df_atual, 100*(df_antes-df_atual)/df_antes))

__Dependendo do percentual podemos deletar as linhas com latitude e longitude nulas.__


__Ou preencher os campos númericos nulos com as médias de cada coluna.__

In [ ]:
# df_furto_veiculos_bimestral_sp = df_furto_veiculos_bimestral_sp.dropna(subset=['LATITUDE', 'LONGITUDE'])

df_furto_veiculos_bimestral_morro_agudo = df_furto_veiculos_bimestral_morro_agudo.fillna(df_furto_veiculos_bimestral_morro_agudo.mean(numeric_only=True))

In [ ]:
df_furto_veiculos_bimestral_morro_agudo.isnull().sum()

__Criando uma coluna geometry__

In [ ]:
df_furto_veiculos_bimestral_morro_agudo['geometry'] = None

for index, row in df_furto_veiculos_bimestral_morro_agudo.iterrows():
    df_furto_veiculos_bimestral_morro_agudo.loc[index, 'geometry'] = Point(row.LONGITUDE, row.LATITUDE)

In [ ]:
df_furto_veiculos_bimestral_morro_agudo

__Criando o GeoDataFrame.__

In [ ]:
mapas_furto_veiculos_bimestral_morro_agudo = gpd.GeoDataFrame(df_furto_veiculos_bimestral_morro_agudo, geometry='geometry')

In [ ]:
mapas_furto_veiculos_bimestral_morro_agudo

__Plotandos os mapas com os dados de furtos de veículos e o mapa do município de Morro Agudo.__

In [ ]:
mapas_furto_veiculos_bimestral_morro_agudo.plot()

In [ ]:
mapas_municipio_morroagudo.plot()

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))

mapas_furto_veiculos_bimestral_morro_agudo.plot(ax=ax)

mapas_municipio_morroagudo.plot(ax=ax, facecolor='None', edgecolor='black')

__Criar um polígono do município de Morro Agudo__

In [ ]:
# Criar um polígono do município de Aparecida

poligono_morro_agudo = mapas_municipio_morroagudo.iloc[0].geometry

In [ ]:
poligono_morro_agudo

__Fazendo a interseção dos mapas de furtos de veículos com o polígono do município de Morro Agudo.__

In [ ]:
mapas_furto_veiculos_bimestral_morro_agudo = mapas_furto_veiculos_bimestral_morro_agudo[mapas_furto_veiculos_bimestral_morro_agudo.intersects(poligono_morro_agudo)]

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

mapas_furto_veiculos_bimestral_morro_agudo.plot(ax=ax)

mapas_municipio_morroagudo.plot(ax=ax, facecolor='None', edgecolor='black')

In [ ]:
mapas_furto_veiculos_bimestral_morro_agudo

In [ ]:
mapas_furto_veiculos_bimestral_morro_agudo_temp = mapas_furto_veiculos_bimestral_morro_agudo[['CIDADE','LATITUDE','LONGITUDE', 'geometry']]

In [ ]:
mapas_furto_veiculos_bimestral_morro_agudo_temp.to_file('dados/furto_veiculos_bimestral_morro_agudo.json', driver='GeoJSON')

In [ ]:
mapas_furto_veiculos_bimestral_morro_agudo_temp

__Criando os mapas do dados do município de Morro Agudo.__

In [ ]:
# Carregando dados do mapa

gdf_geometria_morro_agudo = gpd.read_file('dados/municipio_morro_agudo.json', driver='GeoJSON')

In [ ]:
# Carregando dados de furto

gdf_furto_veiculos_bimestre_morro_agudo = gpd.read_file('dados/furto_veiculos_bimestral_morro_agudo.json', driver='GeoJSON')

In [ ]:
media_latitude = gdf_furto_veiculos_bimestre_morro_agudo['LATITUDE'].mean()

media_longitude = gdf_furto_veiculos_bimestre_morro_agudo['LONGITUDE'].mean()

fmap = folium.Map(location=[media_latitude, media_longitude], tiles='cartodbpositron')

limites = folium.features.GeoJson(gdf_geometria_morro_agudo,
                                 style_function=lambda feature:{
                                     'color':'black',
                                     'weight':2,
                                     'fillOpacity':0.0
                                 })
fmap.add_child(limites)

mc = FastMarkerCluster(gdf_furto_veiculos_bimestre_morro_agudo[['LATITUDE', 'LONGITUDE']])

fmap.add_child(mc)

fmap

__Salvando o mapa em um arquivo html.__

In [ ]:
fmap.save('dados/mapa_furto_veiculos_morro_agudo.html')

#__Fazendo a análise dos furtos das Microrregiões de São Paulo.__

__Será realizado um procedimento semelhante, mas agora iremos analisar a quantidade de furtos de veículos nas Microrregiões do estado de São Paulo.__

In [ ]:
mapas_micro_sp = gpd.read_file('dados_micro/SP_Microrregioes_2022.shp')

In [ ]:
mapas_micro_sp

In [ ]:
mapas_micro_sp.plot(figsize=(6,8), facecolor='white', edgecolor='black')

plt.show()

In [ ]:
mapas_micro_sp.to_file('dados_micro/micro_sp.json', driver='GeoJSON')

In [ ]:
df_01 = pd.read_excel('dados/DadosBO_2022_12_furtos_veiculos.xlsx')
df_02 = pd.read_excel('dados/DadosBO_2023_1_furtos_veiculos.xlsx')

In [ ]:
df_furto_veiculos_bimestral_sp = pd.concat([df_01, df_02])

In [ ]:
df_furto_veiculos_bimestral_sp

In [ ]:
df_furto_veiculos_micro_sp = df_furto_veiculos_bimestral_sp.fillna(df_furto_veiculos_bimestral_sp.mean(numeric_only=True))

In [ ]:
df_furto_veiculos_micro_sp['geometry'] = None

for index, row in df_furto_veiculos_micro_sp.iterrows():
    df_furto_veiculos_micro_sp.loc[index, 'geometry'] = Point(row.LONGITUDE, row.LATITUDE)

In [ ]:
df_furto_veiculos_micro_sp

In [ ]:
mapas_furto_veiculos_micro_sp = gpd.GeoDataFrame(df_furto_veiculos_micro_sp, geometry='geometry')

In [ ]:
mapas_furto_veiculos_micro_sp

In [ ]:
mapas_micro_sp

In [ ]:
mapas_furto_veiculos_micro_sp.plot()

In [ ]:
mapas_micro_sp.plot()

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))

mapas_furto_veiculos_micro_sp.plot(ax=ax)

mapas_micro_sp.plot(ax=ax, facecolor='None', edgecolor='black')

In [ ]:
#poligono_micro_sp = mapas_micro_sp.loc[4].geometry

#poligono_micro_sp_temp = mapas_micro_sp.geometry

for i in mapas_micro_sp.index:
  poligono_micro_sp_temp = poligono_micro_sp_temp.union(mapas_micro_sp.iloc[i].geometry)



In [ ]:
poligono_micro_sp_temp

In [ ]:
mapas_furto_veiculos_micro_sp_temp = mapas_furto_veiculos_micro_sp[mapas_furto_veiculos_micro_sp.intersects(poligono_micro_sp_temp)]

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

mapas_furto_veiculos_micro_sp.plot(ax=ax)

mapas_micro_sp.plot(ax=ax, facecolor='None', edgecolor='black')

In [ ]:

mapas_furto_veiculos_micro_sp_temp = mapas_furto_veiculos_micro_sp[['CIDADE','LATITUDE','LONGITUDE', 'geometry']]

In [ ]:
mapas_furto_veiculos_micro_sp_temp.to_file('dados_micro/furto_veiculos_micro_sp.json', driver='GeoJSON')

In [ ]:
mapas_furto_veiculos_micro_sp_temp

In [ ]:
# Carregando dados do mapa

gdf_geometria_micro_sp = gpd.read_file('dados_micro/micro_sp.json', driver='GeoJSON')

In [ ]:
# Carregando dados de furto

gdf_furto_veiculos_micro_sp = gpd.read_file('dados_micro/furto_veiculos_micro_sp.json', driver='GeoJSON')

In [ ]:
media_latitude = gdf_furto_veiculos_micro_sp['LATITUDE'].mean()

media_longitude = gdf_furto_veiculos_micro_sp['LONGITUDE'].mean()

fmap = folium.Map(location=[media_latitude, media_longitude], tiles='cartodbpositron')

limites = folium.features.GeoJson(gdf_geometria_micro_sp,
                                 style_function=lambda feature:{
                                     'color':'black',
                                     'weight':2,
                                     'fillOpacity':0.0
                                 })
fmap.add_child(limites)

mc = FastMarkerCluster(gdf_furto_veiculos_micro_sp[['LATITUDE', 'LONGITUDE']])

fmap.add_child(mc)

fmap

In [ ]:
fmap.save("dados_micro/furtos_veiculos_microrregioes_sp.html")

In [ ]:
for index, municipio in gdf_geometria_micro_sp.iterrows():
    qtd_roubos = len(gdf_furto_veiculos_micro_sp[gdf_furto_veiculos_micro_sp.intersects(municipio.geometry)])
    gdf_geometria_micro_sp.loc[index, 'qtd_roubos'] = qtd_roubos

In [ ]:
gdf_geometria_micro_sp

In [ ]:
fmap = folium.Map(location=[media_latitude, media_longitude], tiles='cartodbpositron')

for _, municipios in  gdf_geometria_micro_sp.iterrows():

    municipios_geojson = folium.features.GeoJson(municipios.geometry,
                                 style_function=lambda feature:{
                                     'color':'green',
                                     'weight':2,
                                     'fillOpacity':0.1
                                 })
    popup = folium.Popup("""
                            Microrregião: {} <br>
                            Furtos de Veículos Dez/22 e Jan/23: {}
                            """.format(municipios.NM_MICRO, str(int(municipios.qtd_roubos))), max_width=300)

    popup.add_to(municipios_geojson)

    municipios_geojson.add_to(fmap)

fmap

In [ ]:
fmap.save("dados_micro/quantidade_roubos _microrregioes_sp.html")